### 과제
**1. 남자와 여자를 주민번호를 통해 몇 명 있는지 분류하라**  
**2. 가장 많이 쓰는 웹사이트(이메일을 보고 판단)은 무엇인가**  
**3. 가장 나이가 많은 사람과 가장 나이가 어린 사람은 몇 살인가**  
**4. 잘못된 주민번호가 있다면 찾고 이유를 밝혀라**

------

In [7]:
class AutoProcessing:
    
    def __init__(self, line):
        self.__name = line.split()[0]
        self.__tel = line.split()[1]
        self.__id = line.split()[2]
        self.__mail = line.split()[-1]
        
        if self.__id.split("-")[-1][0] in ("9", "1", "3", "5", "7"):
            self.__sex = "Male"

        else:
            self.__sex = "Female"

    def seedata(self):
        print("Name:", self.__name)
        print("Tel.:", self.__tel)
        print("ID:", self.__id)
        print("E-mail:", self.__mail)
        print("Sex:", self.__sex)
        
    def export_to_sql(self):
        sql = """INSERT INTO person(Name, RRN, Tel, Email, Sex)
VALUES ('%s', '%s', '%s', '%s', '%s');""" %(self.__name, self.__id, self.__tel, self.__mail, self.__sex)
        
        return sql

In [1]:
import pymysql as pms

cnx = pms.connect(host = "plen-tutoring.mysql.database.azure.com", 
                  port = 3306, 
                  user = "plentir@plen-tutoring", 
                  password = "yjy990810!", 
                  database = "PyMySQL_test", 
                  charset = "utf8")

cursor = cnx.cursor()

In [3]:
mk_table = """CREATE TABLE person(
ID INT UNSIGNED NOT NULL AUTO_INCREMENT, 
Name VARCHAR(20) NOT NULL, 
RRN VARCHAR(30) NULL, 
Tel VARCHAR(20) NULL, 
Email VARCHAR(30) NULL, 
Sex VARCHAR(10) NULL, 
PRIMARY KEY(ID)
) DEFAULT CHARSET = utf8 AUTO_INCREMENT = 1;
"""

cursor.execute(mk_table)

0

In [2]:
sql = "DROP TABLE person"

cursor.execute(sql)

0

In [8]:
import os

files = os.listdir(os.environ["USERPROFILE"] + "\\desktop\\과제2")

for i in files:
    f = open(os.environ["USERPROFILE"] + "\\desktop\\과제2\\" + i, "r")
    raw_data = f.readline()
    
    a = AutoProcessing(raw_data)
    sql = a.export_to_sql()
    
    cursor.execute(sql)
    
    f.close()

print("end")

cnx.commit()
print("커밋됨.")

cnx.close()

end
커밋됨.


In [63]:
inquiry = """
SELECT *
FROM person
ORDER BY ID DESC
"""

cursor.execute(inquiry)

InterfaceError: (0, '')

In [4]:
def processor(index, splitter = None, target = None, start = 0, end = None):
    import os
    
    prc_data = []
        
    files = os.listdir(os.environ["USERPROFILE"] + "/desktop/과제2/")
    
    if target is None:
        for i in files:
            f = open(os.environ["USERPROFILE"] + "/desktop/과제2/" + i, "r")
            
            raw_data = f.readline().split()
            
            prc_data.append(raw_data[index].split(splitter)[start : end])
            
            f.close()
    
    else:
        for i in files:
            f = open(os.environ["USERPROFILE"] + "/desktop/과제2/" + i, "r")
            
            raw_data = f.readline().split()
            
            prc_data.append(raw_data[index].split(splitter)[target][start : end])
            
            f.close()
    
    return prc_data

In [6]:
def count(key = None):
    
    if key == "sex":  # 남자/여자 인원 수 세기
        n_male = n_female = 0
        
        data = processor(index = -2, splitter = "-", target = -1, end = 1)
        
        for i in data:
            if i in ("9", "1", "3", "5", "7"):
                n_male += 1
            
            else:
                n_female += 1
        
        print("Male:", n_male)
        print("Female:", n_female)
        
        return (n_male, n_female)
                  
    
    elif key == "portals":  # 가장 많이/적게 쓰는 포탈 찾기
        portals = processor(index = -1, splitter = "@", target = -1)

        user_num = {}

        for p in set(portals):
            user_num[p] = portals.count(p)

        temp = tuple(user_num.items())

        _mx = max(temp[n][-1] for n in range(len(temp)))
        _mn = min(temp[n][-1] for n in range(len(temp)))

        for n in range(len(temp)):
            
            if _mx is temp[n][-1]:
                most_used = temp[n][0]

            elif _mn is temp[n][-1]:
                least_used = temp[n][0]

        print("Most used portals: %s (%s)" %(most_used, _mx))
        print("Least used portals: %s (%s)" %(least_used, _mn))
    
        return user_num
                
        
    elif key == "ages":  # 가장 늙은/어린 사람의 나이 찾기
        raw_ages = processor(index = -2, splitter = "-", target = 0, end = 2)
        ages = []
        
        for n in range(len(raw_ages)):
            
            if int(raw_ages[n]) <= 19 and int(raw_ages[n]) >= 0:
                ages.append(19 - int(raw_ages[n]) + 1)
                
            else:
                ages.append(119 - int(raw_ages[n]) + 1)
    
        print("Youngest person: %s years old" %max(ages))
        print("Oldest person: %s years old" %min(ages))
        
        return (max(ages), min(ages))
    
    else:
        print("잘못된 인수를 입력했습니다.")
        
        return 1
    

if __name__ == "__main__":
    count("sex")
    count("portals")
    count("ages")

Male: 10000
Female: 10000
Most used portals: daum.net (3438)
Least used portals: nate.com (3252)
Youngest person: 100 years old
Oldest person: 1 years old


------
### MySQL table 생성하기

    CREATE TABLE testTable(                               (1)
      id INT(11) NOT NULL AUTO_INCREMENT,                 (2)
      name VARCHAR(20) NOT NULL,                          (3)
      ouccupation VARCHAR(20) NULL,                       (4)
      height SMALLINT,                                    (5)
      profile TEXT NULL,                                  (6)
      date  DATETIME,                                     (7)
      CONSTRAINT testTable_PK PRIMARY KEY(id)             (8)
    );
------
(1) 테이블을 생성하는 명령어입니다.  
(2) id라는 컬럼을 추가하는 데, INT 타입으로 지정합니다. NOT NULL은 값을 비워둘 수 없음을 의미합니다. AUTO_INCREMENT는 자동으로 값이 1씩 증가하도록 설정하는 옵션입니다.  
(3) name이라는 컬럼을 추가하는데, 가변길이로 20글자까지 허용합니다. (20글자가 넘어가면 20글자에서 자릅니다.)  
(4) 위와 동일하지만, 값을 비워두는 것을 허용합니다.  
(5) SMALLINT는 INT보다 가질 수 있는 값의 범위가 작습니다. 메모리 측면에서 이득입니다.  
(6) TEXT는 아주 긴 문자열을 취급할 때 사용합니다.  
(7) DATETIME은 날짜와 시간에 관한 타입입니다.  
(8) CONSTRAINT는 제약조건이라는 의미입니다. testTable의 PRIMARY KEY를 id 컬럼으로 지정하겠다는 의미이며, 이 제약조건의 이름을 testTable_PK로 지정한 것입니다.

------
그 외의 옵션으로도 많은 값이 올 수 있습니다.  중요 기능 두 가지만 더 살펴봅시다.  
- CHECK : 값의 범위를 제한합니다.  
- FOREIGN KEY : 다른 테이블과 JOIN할 때 사용합니다.

[출처](https://blog.naver.com/pjok1122/221539169731)

-----

In [1]:
import pymysql as pms

cnx = pms.connect(host = "plen-tutoring.mysql.database.azure.com", 
                  port = 3306, 
                  user = "plentir@plen-tutoring", 
                  password = "yjy990810!", 
                  database = "web_crawling", 
                  charset = "utf8")

cursor = cnx.cursor()

In [4]:
mk_table = """
CREATE TABLE naver_searchkwds(
    ID INT UNSIGNED NOT NULL AUTO_INCREMENT,
    Date VARCHAR(50) NULL,
    Rank SMALLINT NOT NULL,
    Keyword VARCHAR(50) NOT NULL,
    PRIMARY KEY(ID)
) DEFAULT CHARSET = utf8 AUTO_INCREMENT = 1;
"""

cursor.execute(mk_table)

cursor.execute("SHOW TABLES;")

1

In [3]:
name_table = "naver_searchkwds"

cursor.execute("DROP TABLE %s;" %name_table)

0

In [5]:
name_table = "naver_searchkwds"

cursor.execute("DESC %s;" %name_table)

4

In [7]:
inquiry = """
SELECT ID, Date AS 날짜, Rank AS 순위, Keyword AS 검색어
FROM naver_searchkwds
ORDER BY ID DESC
"""

cursor.execute(inquiry)

# result = cursor.fetchall()

0